Projeto Final:\
\
Guia para Legend of Zelda: Breath of the Wild

In [15]:
from plyer import notification
from datetime import datetime
import pandas as pd
import requests
import sqlite3

In [16]:
def alerta(nivel, base, etapa):

    hoje = datetime.now()
    hoje = hoje.strftime("%d/%m/%Y %H:%M:%S")

    message = f'Falha no carregamento da base {base} na etapa {etapa}.\n{hoje}'
    
    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    
    notification.notify(
        title = title,
        message = message,
        app_name = 'alerta',
        timeout = 10)

Banco de Dados

In [17]:
def retrieve_table():
    conn = sqlite3.connect('zelda.db')
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql_query(query, conn)

    conn.close()
    return schema

def save_db(df,table_name):
    conn = sqlite3.connect('zelda.db')
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()
    return True

def load_db(table_name):
    conn = sqlite3.connect('zelda.db')
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)

    conn.close()
    return df

Extração

In [18]:
def get_api(url):
    response = requests.get(url)

    if response.status_code == 200:
        data_json = response.json()
        data_json = data_json['data']
        return data_json
    else:
        print(f'Erro {response.status_code}')

In [19]:
def get_item():
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    id          = [id for id in compendium_df['id']]
    name        = [name for name in compendium_df['name']]
    locations   = [loc for loc in compendium_df['common_locations']]
    drops       = [drop for drop in compendium_df['drops']]

    item_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'drops': drops})
    item_df = item_df.explode('locations', ignore_index=True).explode('drops', ignore_index=True).dropna().sort_values(by='id', ascending=True)
    return item_df

def get_food():
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    id          = [id for id in compendium_df['id']]
    name        = [name for name in compendium_df['name']]
    locations   = [loc for loc in compendium_df['common_locations']]
    effect      = [fx for fx in compendium_df['cooking_effect']]
    heal        = [heal for heal in compendium_df['hearts_recovered']]

    food_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'effect': effect,
        'heal': heal})
    food_df = food_df.explode('locations', ignore_index=True).dropna().sort_values(by='id', ascending=True)
    return food_df

def get_equipment():
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/category/equipment'
    data_json = get_api(url)
    df_equipment = pd.DataFrame(data_json)
    properties = list(df_equipment['properties'])
    df_properties = pd.DataFrame(properties)
    df_equipment = pd.concat([df_equipment, df_properties], axis=1)
    id          = [id for id in df_equipment['id']]
    name        = [name for name in df_equipment['name']]
    locations   = [loc for loc in df_equipment['common_locations']]
    attack      = [atk for atk in df_equipment['attack']]
    defense     = [dfc for dfc in df_equipment['defense']]

    equipment_df = pd.DataFrame({
        'id': id,
        'name': name,
        'locations': locations,
        'attack': attack,
        'defense': defense})
    equipment_df = equipment_df.explode('locations', ignore_index=True).fillna(0).sort_values(by='id', ascending=True)
    return equipment_df

def get_compendium():
    url = 'https://botw-compendium.herokuapp.com/api/v3/compendium/all'
    data_json = get_api(url)
    compendium_df = pd.DataFrame(data_json)
    
    return compendium_df

In [20]:
def extração():
    try:
        item_df = get_item()
        save_db(item_df, 'item')
    except:
        alerta(
            nivel = 3,
            base = 'item',
            etapa = 'extração')
    try:
        food_df = get_food()
        save_db(food_df, 'food')
    except:
        alerta(
            nivel = 3,
            base = 'food',
            etapa = 'extração')
    try:
        equipment_df = get_equipment()
        save_db(equipment_df, 'equipment')
    except:
        alerta(
            nivel = 3,
            base = 'equipment',
            etapa = 'extração')

Tratamento

In [21]:
def tratamento():
    try:
        compendium_df = get_compendium()
        columns_drop = ['dlc', 'image', 'edible']
        compendium_df = compendium_df.drop(columns=columns_drop, axis=1).fillna('')
        columns_astype = ['common_locations', 'drops', 'properties']
        compendium_df[columns_astype] = compendium_df[columns_astype].apply(lambda i: i.astype('str'))
        save_db(compendium_df, 'compendium')
    except:
        alerta(
            nivel = 2,
            base = 'compendium',
            etapa = 'tratamento')

Executar

In [22]:
extração()

In [23]:
tratamento()

Carregar

In [24]:
retrieve_table()

,name
0,item
1,food
2,equipment
3,compendium


In [25]:
load_db('item')

,id,name,locations,drops
0,9,bushy-tailed squirrel,Hyrule Field,acorn
1,9,bushy-tailed squirrel,Hyrule Ridge,acorn
2,10,woodland boar,Hyrule Field,raw meat
3,10,woodland boar,Lanayru Great Spring,raw meat
4,11,red-tusked boar,Akkala Highlands,raw prime meat
...,...,...,...,...
432,388,rare ore deposit,Greater Hyrule,ruby
433,388,rare ore deposit,Greater Hyrule,flint
434,388,rare ore deposit,Greater Hyrule,topaz
435,389,luminous ore deposit,Greater Hyrule,flint


In [26]:
load_db('food')

,id,name,locations,effect,heal
0,48,hyrule bass,West Necluda,,1.0
1,48,hyrule bass,Hyrule Field,,1.0
2,49,hearty bass,Akkala Highlands,extra hearts,2.0
3,49,hearty bass,West Necluda,extra hearts,2.0
4,50,staminoka bass,Hyrule Field,stamina recovery,1.0
...,...,...,...,...,...
134,198,blue nightshade,West Necluda,stealth up,0.0
135,199,silent princess,West Necluda,stealth up,0.0
136,199,silent princess,Hyrule Ridge,stealth up,0.0
137,200,courser bee honey,Tabantha Frontier,stamina recovery,2.0


In [27]:
load_db('equipment')

,id,name,locations,attack,defense
0,201,master sword,0,30.0,0.0
1,202,tree branch,Hyrule Field,2.0,0.0
2,202,tree branch,West Necluda,2.0,0.0
3,203,torch,Great Hyrule Forest,2.0,0.0
4,203,torch,Hyrule Field,2.0,0.0
...,...,...,...,...,...
337,383,lynel shield,Lanayru Great Spring,0.0,30.0
338,384,mighty lynel shield,Lanayru Great Spring,0.0,44.0
339,384,mighty lynel shield,Hyrule Field,0.0,44.0
340,385,savage lynel shield,Hyrule Field,0.0,62.0


In [28]:
load_db('compendium')

,category,common_locations,description,drops,id,name,cooking_effect,hearts_recovered,properties
0,treasure,['Greater Hyrule'],This deposit contains a good deal of precious ...,"['ruby', 'sapphire', 'diamond', 'amber', 'topa...",388,rare ore deposit,,,
1,treasure,['Greater Hyrule'],This deposit contains a good deal of ore. Brea...,"['ruby', 'diamond', 'amber', 'sapphire', 'topa...",387,ore deposit,,,
2,treasure,['Greater Hyrule'],Fortunes untold (potentially) await the lucky ...,['treasures'],386,treasure chest,,,
3,treasure,['Greater Hyrule'],This deposit contains quite a bit of luminous ...,"['luminous', 'flint']",389,luminous ore deposit,,,
4,monsters,"['Eldin Mountains', 'Tabantha Frontier']",A spirit of fire has taken the form of this gi...,,153,dinraal,,,
...,...,...,...,...,...,...,...,...,...
384,creatures,['Satori Mountain'],They may look like mere rabbits at a fleeting ...,,29,blupee,,,
385,creatures,"['Hebra Mountains', 'Gerudo Highlands']","This breed of maraudo wolf lives in snowy, mou...","['raw prime meat', 'raw gourmet meat']",23,cold-footed wolf,,,
386,creatures,"['East Necluda', 'Akkala Highlands']","Originally native to the mountainous regions, ...",[],18,highland sheep,,,
387,creatures,"['Lanayru Wetlands', 'Akkala Highlands']","These wild cows come equipped with big, strong...","['raw prime meat', 'raw gourmet meat']",16,water buffalo,,,
